In [1]:
import cffi
import numpy as np
import ufl
from dolfinx import fem, jit, mesh
from dolfinx.fem import petsc
from mpi4py import MPI
from petsc4py import PETSc

In [2]:
delta = 1.0
gamma = 0.5
nx, ny = 2, 2

In [3]:
domain = mesh.create_rectangle(
    MPI.COMM_WORLD,
    [np.array([0, 0]), np.array([1, 1])],
    [nx, ny],
    mesh.CellType.triangle,
)

In [4]:
function_space = fem.functionspace(domain, ("Lagrange", 1))
trial_function = ufl.TrialFunction(function_space)
test_function = ufl.TestFunction(function_space)

mass_matrix_term = ufl.inner(trial_function, test_function) * ufl.dx
stiffness_matrix_term = ufl.inner(ufl.grad(trial_function), ufl.grad(test_function)) * ufl.dx
spde_matrix_term = delta * mass_matrix_term + gamma * stiffness_matrix_term

mass_matrix = petsc.assemble_matrix(fem.form(mass_matrix_term))
stiffness_matrix = petsc.assemble_matrix(fem.form(stiffness_matrix_term))
mass_matrix.assemble()
stiffness_matrix.assemble()

In [5]:
dense_mass_matrix_automatic = mass_matrix.convert("dense")
dense_mass_array_automatic = dense_mass_matrix_automatic.getDenseArray()

In [6]:
local_vertex_coordinates = domain.geometry.x
local_cell_vertex_indices = domain.geometry.dofmap
dof_map = function_space.dofmap
pproc_dof_distribution_map = dof_map.index_map
pproc_cell_distribution_map = domain.topology.index_map(domain.topology.dim)

num_local_cells = pproc_cell_distribution_map.size_local
num_global_cells = pproc_cell_distribution_map.size_global
num_cell_dofs = dof_map.dof_layout.num_dofs
num_global_dofs = pproc_dof_distribution_map.size_global

In [7]:
mass_matrix_compiled, *_ = jit.ffcx_jit(
    domain.comm,
    mass_matrix_term,
    form_compiler_options={"scalar_type": PETSc.ScalarType},
)
compiled_kernel = getattr(
    mass_matrix_compiled.form_integrals[0],
    f"tabulate_tensor_{np.dtype(PETSc.ScalarType).name}",
)

ffi = cffi.FFI()

In [9]:
block_diagonal_matrix = PETSc.Mat().createAIJ(
    [num_global_cells * num_cell_dofs, num_global_cells * num_cell_dofs], comm=domain.comm
)
block_diagonal_matrix.setPreallocationNNZ(num_cell_dofs)
block_diagonal_matrix.setUp()

local_global_dof_matrix = PETSc.Mat().createAIJ(
    [num_global_cells * num_cell_dofs, num_global_dofs], comm=domain.comm
)
local_global_dof_matrix.setPreallocationNNZ(1)
local_global_dof_matrix.setUp()

In [10]:
local_cell_inds = np.arange(num_local_cells, dtype=np.int64)
global_cell_inds = pproc_cell_distribution_map.local_to_global(local_cell_inds)

for local_ind, global_ind in zip(local_cell_inds, global_cell_inds, strict=True):
    local_cell_dofs = dof_map.cell_dofs(local_ind)
    global_cell_dofs = pproc_dof_distribution_map.local_to_global(local_cell_dofs).astype(
        PETSc.IntType
    )
    cell_vertex_inds = local_cell_vertex_indices[local_ind]
    cell_vertex_coordinates = local_vertex_coordinates[cell_vertex_inds]
    cell_matrix = np.zeros((num_cell_dofs, num_cell_dofs), dtype=PETSc.ScalarType)

    compiled_kernel(
        ffi.from_buffer(cell_matrix),  # A - output matrix
        ffi.NULL,  # w - coefficient values (none for this form)
        ffi.NULL,  # c - constants (none for this form)
        ffi.from_buffer(cell_vertex_coordinates),  # coordinate_dofs
        ffi.NULL,  # entity_local_index (cell integral)
        ffi.NULL,  # cell_orientation
    )
    cell_matrix = np.linalg.cholesky(cell_matrix)

    block_diag_matrix_inds = np.arange(
        global_ind * num_cell_dofs, (global_ind + 1) * num_cell_dofs, dtype=PETSc.IntType
    )
    block_diagonal_matrix.setValues(
        block_diag_matrix_inds,
        block_diag_matrix_inds,
        cell_matrix,
        addv=PETSc.InsertMode.INSERT_VALUES,
    )

    local_global_map_rows = block_diag_matrix_inds
    local_global_map_cols = global_cell_dofs
    local_global_dof_values = np.ones((num_cell_dofs,), dtype=PETSc.ScalarType)

    for row_ind, col_ind, value in zip(
        local_global_map_rows, local_global_map_cols, local_global_dof_values, strict=True
    ):
        local_global_dof_matrix.setValues(
            row_ind,
            col_ind,
            value,
            addv=PETSc.InsertMode.INSERT_VALUES,
        )

In [11]:
block_diagonal_matrix.assemble()
local_global_dof_matrix.assemble()
local_global_dof_matrix_transposed = local_global_dof_matrix.duplicate(copy=True)
local_global_dof_matrix_transposed.transpose()

mass_matrix_factor = local_global_dof_matrix_transposed.matMult(block_diagonal_matrix)
mass_matrix_factor_transposed = mass_matrix_factor.duplicate(copy=True)
mass_matrix_factor_transposed.transpose()

mass_matrix = mass_matrix_factor.matMult(mass_matrix_factor_transposed)
mass_matrix.assemble()
np.allclose(mass_matrix.getValues(range(9), range(9)), dense_mass_array_automatic)

True